**LOADING REQUIRED LIBRARIES**

In [0]:
import h2o
from sklearn.model_selection import train_test_split,GridSearchCV
import seaborn as sns
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
import math
import pandas as pd
import category_encoders as ce
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM (build 25.152-b12, mixed mode)56-b12)
  Starting server from C:\Users\Abhishek\Anaconda3_NEW\h2o_jar\h2o.jar
  Ice root: C:\Users\Abhishek\AppData\Local\Temp\tmpkpr92ik9
  JVM stdout: C:\Users\Abhishek\AppData\Local\Temp\tmpkpr92ik9\h2o_Abhishek_started_from_python.out
  JVM stderr: C:\Users\Abhishek\AppData\Local\Temp\tmpkpr92ik9\h2o_Abhishek_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.2
H2O cluster version age:,1 year and 12 days !!!
H2O cluster name:,H2O_from_python_Abhishek_qfahk6
H2O cluster total nodes:,1
H2O cluster free memory:,3.533 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


**IMPORTING SCORES DATA FILE**

In [0]:
df = h2o.import_file('C:\AI and neural networks\Assignment 3\API_Scores.csv')
df.head()

Parse progress: |█████████████████████████████████████████████████████████| 100%


Azure_Score,AWS_Score,Google_Score,IBM_Score,Manual_label
0.887542,0.5,0,0.987951,1
0.832603,0.6,0.1,0.655981,1
0.5,0.5,0.1,0,1
0.792194,0.5,0.1,0.580306,1
0.749581,0.5,0.1,0.683194,2
0.5,0.5,0,0.427012,1
0.5,0.5,0,0.990295,1
0.5,0.5,-0.2,-0.24186,1
0.5,0.5,-0.1,-0.516043,1
0.5,0.5,0.3,0,2


In [0]:
df['Manual_label'] = df['Manual_label'].asfactor()

In [0]:
#Setting target variable
y = 'Manual_label'

In [0]:
def get_independent_variables(df, targ):
    C = [name for name in df.columns if name != targ]
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in C:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    x=ints+enums+reals
    return x

In [0]:
X=get_independent_variables(df, y) 
print(X)

['Azure_Score', 'AWS_Score', 'Google_Score', 'IBM_Score']


**FINDING BEST MODEL**

In [0]:
aml = H2OAutoML(max_models = 30, max_runtime_secs= 3600, seed = 1)
aml.train(x = X, y = y, training_frame = df)

AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


**MODEL PERFORMANCE**

In [0]:
aml.leader

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_0_AutoML_20190320_212351


ModelMetricsMultinomial: drf
** Reported on train data. **

MSE: 0.32266044167935154
RMSE: 0.5680320780372808
LogLoss: 2.53022840617711
Mean Per-Class Error: 0.5534829563806847
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



0,1,2,Error,Rate
20.0,44.0,60.0,0.8387097,104 / 124
26.0,285.0,224.0,0.4672897,250 / 535
28.0,207.0,428.0,0.3544495,235 / 663
74.0,536.0,712.0,0.4455371,"589 / 1,322"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.5544629
2,0.8835098
3,1.0



ModelMetricsMultinomial: drf
** Reported on validation data. **

MSE: 0.3196094466809729
RMSE: 0.5653401159310852
LogLoss: 1.5390621185899116
Mean Per-Class Error: 0.5929475015812776
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



...

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
accuracy,0.5809434,0.0076685,0.5773585,0.5773585,0.5719697,0.5757576,0.6022728
err,0.4190566,0.0076685,0.4226415,0.4226415,0.4280303,0.4242424,0.3977273
err_count,110.8,2.0688162,112.0,112.0,113.0,112.0,105.0
logloss,1.444977,0.1811897,1.2535964,1.6045083,1.0346824,1.6579013,1.6741967
max_per_class_error,0.8445042,0.0574736,0.84,0.8518519,0.6956522,0.9090909,0.9259259
mean_per_class_accuracy,0.4662655,0.0122807,0.4628572,0.4654287,0.4968604,0.4427793,0.4634018
mean_per_class_error,0.5337345,0.0122807,0.5371429,0.5345713,0.5031396,0.5572207,0.5365982
mse,0.3122305,0.0085228,0.3082426,0.3303622,0.3094557,0.3189836,0.2941082
r2,0.2707393,0.0298693,0.2855995,0.2470079,0.2605656,0.2176004,0.3429232
rmse,0.5586717,0.0076277,0.555196,0.5747715,0.5562874,0.5647864,0.5423175


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,validation_rmse,validation_logloss,validation_classification_error
,2019-03-20 21:23:52,1.015 sec,0.0,nan,nan,nan,nan,nan,nan
,2019-03-20 21:23:52,1.021 sec,1.0,0.6906099,14.6154448,0.4819533,0.6938317,14.8045989,0.5030488
,2019-03-20 21:23:52,1.026 sec,2.0,0.6864998,13.8562690,0.4820513,0.6402473,9.9654017,0.4603659
,2019-03-20 21:23:52,1.032 sec,3.0,0.6756358,13.0468137,0.4799176,0.5956584,6.7087240,0.4420732
,2019-03-20 21:23:52,1.037 sec,4.0,0.6584804,11.7472301,0.4609091,0.5964814,5.4467096,0.4420732
---,---,---,---,---,---,---,---,---,---
,2019-03-20 21:23:52,1.163 sec,19.0,0.5702338,2.9596094,0.4409985,0.5681519,1.8180369,0.4420732
,2019-03-20 21:23:52,1.173 sec,20.0,0.5693695,2.7431057,0.4440242,0.5676997,1.7232332,0.4390244
,2019-03-20 21:23:52,1.186 sec,21.0,0.5689346,2.7187892,0.4455371,0.5667711,1.6353826,0.4512195
,2019-03-20 21:23:52,1.197 sec,22.0,0.5675688,2.6447551,0.4440242,0.5668134,1.6370852,0.4481707



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
IBM_Score,5458.5761719,1.0,0.5296398
Azure_Score,1956.8504639,0.3584910,0.1898711
Google_Score,1672.5225830,0.3064027,0.1622831
AWS_Score,1218.2558594,0.2231820,0.1182061


In [0]:
aml.leaderboard

model_id,mean_per_class_error
DRF_0_AutoML_20190320_212351,0.534463
DRF_0_AutoML_20190320_211601,0.534463
XRT_0_AutoML_20190320_212351,0.542891
XRT_0_AutoML_20190320_211601,0.542891
GBM_grid_0_AutoML_20190320_211601_model_5,0.544317
GBM_grid_0_AutoML_20190320_212351_model_5,0.544317
GBM_grid_0_AutoML_20190320_211601_model_3,0.551595
GBM_grid_0_AutoML_20190320_212351_model_3,0.551595
StackedEnsemble_AllModels_0_AutoML_20190320_211601,0.553711
GBM_grid_0_AutoML_20190320_211601_model_2,0.555178


In [0]:
model = h2o.get_model(aml.leaderboard[0,'model_id'])
model.algo

'drf'